## **<font color="red">Rewind the Session</font>**
- When you rewind a session, you specify a user request, or ***invocation***, that you want to undo, and the system undoes that request and the requests after it.
- What `rewind_async()` ACTUALLY Does
  - `runner.rewind_async()` **DOES NOT**:
    - ❌ Reset the model’s brain
    - ❌ Recompute past model responses
    - ❌ Automatically change chat-based memory
  - It **DOES**:
    - ✅ Delete all events AFTER a specific invocation
    - ✅ Restore `session.state` to what it was at that moment
    - ✅ Restore the event timeline
    - ✅ Reset structured memory

### Understanding Rewind in Google ADK (In-Memory Sessions)

- When working with `Runner.rewind_async()` in Google ADK, it is important to understand what rewind actually does — and what it does NOT do.
- Many developers initially think rewind behaves like an "undo" operation.

**It does not.**

**Rewind performs timeline truncation.**

---

## Mental Model: Invocation Timeline

Assume we have the following sequence of agent calls:
```
A → color = red
B → color = blue
C → color = yellow
```

This represents three invocations stored in the session timeline.

---

## What Does `rewind_before_invocation_id` Do?

If we execute `rewind_before_invocation_id = B`, it means:

> "Restore the session to the moment just BEFORE invocation B happened."

**Result:**

- Invocation B is deleted
- Invocation C is deleted
- Invocation A is preserved

The new timeline becomes:
```
A → color = red
```

And the current session state becomes:
```python
{"color": "red"}
```

---

## What Rewind Does NOT Do

Rewind does **NOT**:

- Undo only one step
- Reverse history
- Keep future state while deleting past
- Recalculate model responses
- Selectively delete middle invocations

**There is no `rewind_after_invocation_id`.**

Rewind always removes everything after the specified invocation.

---

## Think of It Like Git

If your timeline is:
```
A → B → C
```

Rewind before B behaves like:
```bash
git reset --hard A
```

Everything after A is deleted.

---

## What Happens Next?

After rewind, if you make a new call:
```
A (red)
D (green)
```

The timeline becomes:
```
A → D
```

The previous B (blue) and C (yellow) are **permanently gone**.

This creates a new forward branch from A.

---

## Key Takeaway

**Rewind in ADK is a deterministic timeline cut operation.**

It restores:
- Session state
- Invocation history
- Tool execution graph

to exactly how they were **before** the specified invocation.

**It is not an undo.**  
**It is time travel with truncation.**

> **Best Practice:** If you want rewind to behave predictably, always use structured `state_delta` updates rather than relying only on conversational memory.

In [8]:
import os
import asyncio

from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

from config import config  # must contain GOOGLE_API_KEY


# -------------------------------------------------
# Configuration
# -------------------------------------------------
os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

APP_NAME = "my_app"
USER_ID = "user-1"

SYSTEM_INSTRUCTION = """
You are a helpful Intelligence Assistant.
Acknowledge state updates.
"""


# -------------------------------------------------
# Helper: Create User Message
# -------------------------------------------------
def user_message(text: str) -> types.Content:
    return types.Content(
        role="user",
        parts=[types.Part(text=text)]
    )


# -------------------------------------------------
# Helper: Print Model Text
# -------------------------------------------------
def print_model_response(events):
    for e in events:
        if hasattr(e, "content") and e.content:
            for part in e.content.parts:
                if hasattr(part, "text") and part.text:
                    print("MODEL:", part.text)


# -------------------------------------------------
# Helper: Print Session State
# -------------------------------------------------
async def print_state(session_service, session_id):
    session_obj = await session_service.get_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=session_id,
    )
    print("CURRENT SESSION STATE:", session_obj.state)


# -------------------------------------------------
# Main Async Function
# -------------------------------------------------
async def main():

    # 1️⃣ Create Agent
    agent = Agent(
        name="intelligence_agent",
        model="gemini-2.5-flash",
        description="AI agent for helpful intelligence and analytics.",
        instruction=SYSTEM_INSTRUCTION,
    )

    # 2️⃣ Session Service
    session_service = InMemorySessionService()

    # 3️⃣ Runner
    runner = Runner(
        agent=agent,
        app_name=APP_NAME,
        session_service=session_service,
    )

    # 4️⃣ Create Session
    session = await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
    )

    session_id = session.id

    print(f"\nSession created: {session_id}")
    print("=" * 60)

    # -------------------------------------------------
    # A → RED
    # -------------------------------------------------
    print("A: Setting color = red")

    events_A = []
    async for event in runner.run_async(
        user_id=USER_ID,
        session_id=session_id,
        new_message=user_message("set color to red"),
        state_delta={"color": "red"},  # REAL STATE
    ):
        events_A.append(event)

    print_model_response(events_A)
    await print_state(session_service, session_id)
    print("-" * 60)

    # -------------------------------------------------
    # B → BLUE
    # -------------------------------------------------
    print("B: Setting color = blue")

    events_B = []
    async for event in runner.run_async(
        user_id=USER_ID,
        session_id=session_id,
        new_message=user_message("set color to blue"),
        state_delta={"color": "blue"},
    ):
        events_B.append(event)

    print_model_response(events_B)
    await print_state(session_service, session_id)
    print("-" * 60)

    # -------------------------------------------------
    # C → YELLOW
    # -------------------------------------------------
    print("C: Setting color = yellow")

    events_C = []
    async for event in runner.run_async(
        user_id=USER_ID,
        session_id=session_id,
        new_message=user_message("set color to yellow"),
        state_delta={"color": "yellow"},
    ):
        events_C.append(event)

    print_model_response(events_C)
    await print_state(session_service, session_id)
    print("-" * 60)

    # -------------------------------------------------
    # D → GREEN
    # -------------------------------------------------
    print("D: Setting color = green")

    events_D = []
    async for event in runner.run_async(
        user_id=USER_ID,
        session_id=session_id,
        new_message=user_message("set color to green"),
        state_delta={"color": "green"},
    ):
        events_D.append(event)

    print_model_response(events_D)
    await print_state(session_service, session_id)
    print("=" * 60)

    # -------------------------------------------------
    # REWIND BEFORE C (Delete C and D)
    # -------------------------------------------------
    invocation_events_C = [
        e for e in events_C if hasattr(e, "invocation_id")
    ]

    if invocation_events_C:
        rewind_id = invocation_events_C[0].invocation_id

        print(f"\nRewinding BEFORE C (Invocation: {rewind_id})")

        await runner.rewind_async(
            user_id=USER_ID,
            session_id=session_id,
            rewind_before_invocation_id=rewind_id,
        )

        print("Rewind completed.")

        print("\nSTATE AFTER REWIND:")
        await print_state(session_service, session_id)

    else:
        print("Could not find invocation_id for C.")


# -------------------------------------------------
# RUN (Jupyter Safe)
# -------------------------------------------------
await main()



Session created: ded1920d-0467-4383-b0ac-b92d6d83d9a2
A: Setting color = red
MODEL: Okay, I've noted your preference for red.
CURRENT SESSION STATE: {'color': 'red'}
------------------------------------------------------------
B: Setting color = blue
MODEL: Okay, I've updated the color to blue.
CURRENT SESSION STATE: {'color': 'blue'}
------------------------------------------------------------
C: Setting color = yellow
MODEL: Okay, I've updated the color to yellow.
CURRENT SESSION STATE: {'color': 'yellow'}
------------------------------------------------------------
D: Setting color = green
MODEL: Okay, I've updated the color to green.
CURRENT SESSION STATE: {'color': 'green'}

Rewinding BEFORE C (Invocation: e-32738179-bffa-4cf6-b4d2-1a9edd0ab936)
Rewind completed.

STATE AFTER REWIND:
CURRENT SESSION STATE: {'color': 'blue'}


- A → red
- B → blue
- C → yellow   ❌ deleted
- D → green    ❌ deleted
